In [ ]:
!pip install transformers datasets torch accelerate huggingface_hub

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

dataset = load_dataset("imdb")

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

trainer.train()


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-ca4134e7c4d1>:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wand

Epoch,Training Loss,Validation Loss
1,No log,0.304949
2,No log,0.290772
3,No log,0.339065


TrainOutput(global_step=375, training_loss=0.2845325724283854, metrics={'train_runtime': 20840.0911, 'train_samples_per_second': 0.288, 'train_steps_per_second': 0.018, 'total_flos': 789333166080000.0, 'train_loss': 0.2845325724283854, 'epoch': 3.0})

In [1]:
from huggingface_hub import login

# Login to Hugging Face Hub
login()

# Push model and tokenizer to Hugging Face
model.push_to_hub("imdb-sentiment-classifier")
tokenizer.push_to_hub("imdb-sentiment-classifier")


NameError: name 'model' is not defined

In [ ]:
from transformers import pipeline

# Load model from Hugging Face Hub
sentiment_classifier = pipeline("text-classification", model="your-username/imdb-sentiment-classifier")

# Test the model
result = sentiment_classifier("I loved the movie, it was fantastic!")
print(result)


OSError: your-username/imdb-sentiment-classifier is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`